<a href="https://colab.research.google.com/github/barry05/BCI/blob/master/Arousal_Valence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. INSTALL NEEDED LIBRARIES (pyeeg)

In [ ]:
pip install git+https://github.com/forrestbao/pyeeg.git

  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-bfl9t63b
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-bfl9t63b
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28122 sha256=e9394965e3f3d985f5c29887d6df004ad5fec4dc748d980decbaefe66fc42e1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-m5nzxx2z/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg


2. IMPORT NEEDED LIBRARIES


In [ ]:
import numpy as np

import pickle as pickle
import pandas as pd
import pyeeg as pe
import seaborn as sns
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

import math

from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

import os
#import tensorflow as tf
import time
from numpy import load

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


3. MOUNT GOOGLE DRIVE (to access the DEAP Dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


4. SET STATIC VARIABLES (channels, bands, sample rate, subjectList, etc.)

In [ ]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01']
#subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16']
#List of subjects, use full 32 if computing power allows

5. DEFINE THE FUNCTION FOR THE FFT PREPROCESSING OF THE DATA

In [ ]:
def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open('/content/drive/My Drive/BCI/ABCI/data_preprocessed_python/s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,39):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec, creating 2sec epoch?
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
                
        meta = np.array(meta)
        np.save('/content/drive/My Drive/BCI/ABCI/data_sampled_at_70/s' + sub, meta, allow_pickle=True, fix_imports=True)



6. DEFINE THE FUNCTION FOR COMPUTING THE ACCURACY OF THE REGRESSION MODELS


In [ ]:
def testing (M, L, model):
    '''
    arguments:  M: testing dataset
                L: testing dataset label
                model: scikit-learn model

    return:     void
    '''
    output = model.predict(M[0:78080:32])
    label = L[0:78080:32]

    k = 0
    l = 0

    for i in range(len(label)):
        k = k + (output[i] - label[i])*(output[i] - label[i]) #square difference 

        #a good guess
        if (output[i] > 5 and label[i] > 5):
            l = l + 1
        elif (output[i] < 5 and label[i] <5):
            l = l + 1

    print ("l2 error:", k/len(label), "classification accuracy:", l / len(label),l, len(label))

7. CREATE EEG DATA FILE AND LABEL FILE FROM ONLY 1 PARTICIPANT


In [ ]:
data_one= []
label_one = []
for subjects in subjectList:
  

    with open('/content/drive/My Drive/BCI/ABCI/s' + subjects + '.npy', 'rb') as file:
        sub = np.load(file,allow_pickle=True)
        for i in range (0,sub.shape[0]):
          data_one.append(sub[i][0])
          label_one.append(sub[i][1])
np.save('data_one', np.array(data_one), allow_pickle=True, fix_imports=True)
np.save('label_one', np.array(label_one), allow_pickle=True, fix_imports=True)

In [ ]:
df=pd.DataFrame(data=data_one)
df.to_csv("data_one.csv",index=False)

df1=pd.DataFrame(data=label_one)
df1.to_csv("label_one.csv",index=False)


datas_one=pd.read_csv("/content/data_one.csv")
#data1

labels_one=pd.read_csv("/content/label_one.csv")
#label1

In [ ]:
print(datas_one.shape)
#print(datas_one)
print(labels_one.shape)
#print(labels_one)

(19032, 70)
(19032, 4)


8. VISUALIZE THE DATASET

In [ ]:
 #names = ['valence', 'arousal', 'dominance', 'liking']
 #eeg_dataset = datas_one.transpose().reindex()

In [ ]:
# Summary of dataset
#print('Summary of dataset\n')
# shape
#print('1. Shape is:')
#print(eeg_dataset.shape)
# head
#print('\n2. First 8 rows are as:')
#print(eeg_dataset.head(8))
# descriptions
#print('\n3. Statistical description:')
#print(eeg_dataset.describe())
#types = eeg_dataset.dtypes
#print('\n4. Data Types:')
#print(types)

In [ ]:
# Data visualisation
# box and whisker plots
#for i in range(5):
#  sns.set_style('whitegrid')
#  sns.boxplot(eeg_dataset[i])
#  plt.title('Magnitude versus ' + str(i+1) + ' channel')
#  plt.show()

In [ ]:
#for i in range(5):
#  sns.distplot(eeg_dataset[i])
#  plt.title('Magnitude versus ' + str(i+1) + ' channel')
#  plt.show()

8. DIVIDE LABELS BETWEEN VALENCE, AROUSAL, DOMINANCE, LIKING

In [ ]:
y_val_one=labels_one.loc[:,'0']
y_ars_one=labels_one.loc[:,'1']
y_lik_one=labels_one.loc[:,'2']
y_dom_one=labels_one.loc[:,'3']

print(y_val_one.shape)
print(y_ars_one.shape)
print(y_lik_one.shape)
print(y_dom_one.shape)


(19032,)
(19032,)
(19032,)
(19032,)


In [ ]:
print(y_val_one)

0        7.71
1        7.71
2        7.71
3        7.71
4        7.71
         ... 
19027    3.24
19028    3.24
19029    3.24
19030    3.24
19031    3.24
Name: 0, Length: 19032, dtype: float64


9. CHANGE LABELS (1-9) TO DISCRETE [(0,1),(0,1,2),(Low,neutral,high)]


9.1 TERTIARY (0, 1, 2)

In [ ]:
#VALENCE tertiary 
val_conditions  = [ y_val_one <= 3.5, (y_val_one < 5.5) & (y_val_one > 3.5), y_val_one >= 5.5 ]
choices     = [0, 1, 2]
#choices     = [ "low","medium","high"]

y_val_ter = np.select(val_conditions, choices, default=np.nan)

#AROUSAL tertiary 
ars_conditions  = [ y_ars_one <= 3.5, (y_ars_one < 5.5) & (y_ars_one > 3.5), y_ars_one >= 5.5 ]

y_ars_ter = np.select(ars_conditions, choices, default=np.nan)

#LIKING tertiary 
lik_conditions  = [ y_lik_one <= 3.5, (y_lik_one < 5.5) & (y_lik_one > 3.5), y_lik_one >= 5.5 ]

y_lik_ter = np.select(lik_conditions, choices, default=np.nan)

#DOMINANCE tertiary 
dom_conditions  = [ y_dom_one <= 3.5, (y_dom_one < 5.5) & (y_dom_one > 3.5), y_dom_one >= 5.5 ]

y_dom_ter = np.select(dom_conditions, choices, default=np.nan)

9.2 BINARY (0,1)

In [ ]:
#VALENCE binary string
val_conditions = [y_val_one <= 4.49, y_val_one >= 4.5]
choices     = [ 0, 1]

y_val_binary = np.select(val_conditions, choices, default=np.nan)

#AROUSAL binary string
ars_conditions  = [y_ars_one <= 4.49, y_ars_one >= 4.5]

y_ars_binary = np.select(ars_conditions, choices, default=np.nan)

#LIKING binary  string
lik_conditions  = [y_lik_one <= 4.49, y_lik_one >= 4.5]

y_lik_binary = np.select(lik_conditions, choices, default=np.nan)

#DOMINANCE binary string
dom_conditions  = [y_dom_one <= 4.49, y_dom_one >= 4.5]

y_dom_binary = np.select(dom_conditions, choices, default=np.nan)

In [ ]:
print(y_dom_one)
print(y_dom_binary)

0        7.83
1        7.83
2        7.83
3        7.83
4        7.83
         ... 
19027    6.15
19028    6.15
19029    6.15
19030    6.15
19031    6.15
Name: 3, Length: 19032, dtype: float64
[1. 1. 1. ... 1. 1. 1.]


11. CLASSIFICATION PROBLEMS for Binary [0,1] and Tertiary [0,1,2]
 * 11.1 SVM
 * 11.2 SVM + PCA
 * 11.3 KNN
 * 11.4 KNN + PCA

11.1 SVM

11.1.1 BINARY [0,1]

11.1.1.1 SVM Binary [0,1] **VALENCE**

In [ ]:
eeg_train, eeg_test, y_val_binary_train, y_val_binary_test = train_test_split(datas_one, y_val_binary, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVC Classifier on TERTIARY VALENCE for ONE Subject
svc_clf_val_binary = SVC(kernel = 'rbf', random_state = 42)
svc_clf_val_binary.fit(eeg_train, y_val_binary_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_predicted_binary_valence_score_SVC = svc_clf_val_binary.predict(eeg_test)
cm = confusion_matrix(y_val_binary_test, y_predicted_binary_valence_score_SVC)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Binary VALENCE using SVC: ")
print(accuracy_score(y_val_binary_test, y_predicted_binary_valence_score_SVC)*100)

[[1803  159]
 [ 104 1741]]
Accuracy score FOR SUBJECT 1 of Binary VALENCE using SVC: 
93.0916732335172


In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_val_binary_test, y_predicted_binary_valence_score_SVC, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.95      0.92      0.93      1962
     class 1       0.92      0.94      0.93      1845

    accuracy                           0.93      3807
   macro avg       0.93      0.93      0.93      3807
weighted avg       0.93      0.93      0.93      3807



11.1.1.1 SVM Binary [0,1] **AROUSAL**

In [ ]:
eeg_train, eeg_test, y_ars_binary_train, y_ars_binary_test = train_test_split(datas_one, y_ars_binary, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVC Classifier on BINARY AROUSAL for ONE Subject
svc_clf_ars_binary = SVC(kernel = 'rbf', random_state = 42)
svc_clf_ars_binary.fit(eeg_train, y_ars_binary_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_predicted_binary_arousal_score_SVC = svc_clf_ars_binary.predict(eeg_test)
cm = confusion_matrix(y_ars_binary_test, y_predicted_binary_arousal_score_SVC)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Binary AROUSAL using SVC: ")
print(accuracy_score(y_ars_binary_test, y_predicted_binary_arousal_score_SVC)*100)

[[1373  119]
 [  66 2249]]
Accuracy score FOR SUBJECT 1 of Binary AROUSAL using SVC: 
95.14053060152351


In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_ars_binary_test, y_predicted_binary_arousal_score_SVC, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.95      0.92      0.94      1492
     class 1       0.95      0.97      0.96      2315

    accuracy                           0.95      3807
   macro avg       0.95      0.95      0.95      3807
weighted avg       0.95      0.95      0.95      3807



11.1.2 TERTIARY [0,1,2]

11.1.2.1 SVM Tertiary [0,1,2] **VALENCE**

In [ ]:
eeg_train, eeg_test, y_val_tertiary_train, y_val_tertiary_test = train_test_split(datas_one, y_val_ter, test_size = 0.2, random_state = 42)

In [ ]:
#Train SVC Classifier on TERTIARY VALENCE for ONE Subject
svc_clf_val_tertiary = SVC(kernel = 'rbf', random_state = 42)
svc_clf_val_tertiary.fit(eeg_train, y_val_tertiary_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_predicted_tertiary_valence_score_SVC = svc_clf_val_tertiary.predict(eeg_test)
cm = confusion_matrix(y_val_tertiary_test, y_predicted_tertiary_valence_score_SVC)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Tertiary Valence using SVC: ")
print(accuracy_score(y_val_tertiary_test, y_predicted_tertiary_valence_score_SVC)*100)

[[1396    1  153]
 [  62  317  112]
 [ 182    0 1584]]
Accuracy score FOR SUBJECT 1 of Tertiary Valence using SVC: 
86.60362490149724


In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_val_tertiary_test, y_predicted_tertiary_valence_score_SVC, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.90      0.88      1550
     class 1       1.00      0.65      0.78       491
     class 2       0.86      0.90      0.88      1766

    accuracy                           0.87      3807
   macro avg       0.90      0.81      0.85      3807
weighted avg       0.87      0.87      0.86      3807



11.1.2.2 SVM Tertiary [0,1,2] **AROUSAL**

In [ ]:
eeg_train, eeg_test, y_ars_tertiary_train, y_ars_tertiary_test = train_test_split(datas_one, y_ars_ter, test_size = 0.2, random_state = 42)

In [ ]:
#Train SVC Classifier on TERTIARY AROUSAL for ONE Subject
svc_clf_ars_tertiary = SVC(kernel = 'rbf', random_state = 42)
svc_clf_ars_tertiary.fit(eeg_train, y_ars_tertiary_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
y_predicted_tertiary_arousal_score_SVC = svc_clf_ars_tertiary.predict(eeg_test)
cm = confusion_matrix(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_SVC)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Tertiary AROUSAL using SVC: ")
print(accuracy_score(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_SVC)*100)

[[ 878    9  206]
 [  32  406   52]
 [  79   16 2129]]
Accuracy score FOR SUBJECT 1 of Tertiary AROUSAL using SVC: 
89.65064355135277


In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_SVC, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.89      0.80      0.84      1093
     class 1       0.94      0.83      0.88       490
     class 2       0.89      0.96      0.92      2224

    accuracy                           0.90      3807
   macro avg       0.91      0.86      0.88      3807
weighted avg       0.90      0.90      0.90      3807



11.2 KNN

11.2.1 KNN Binary [0,1]

11.2.1 KNN Binary [0,1] **VALENCE**


In [ ]:
eeg_train, eeg_test, y_val_binary_train, y_val_binary_test = train_test_split(datas_one, y_val_binary, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVC Classifier on TERTIARY VALENCE for ONE Subject
knn_clf_val_binary = KNeighborsClassifier(n_neighbors=1)
knn_clf_val_binary.fit(eeg_train, y_val_binary_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
y_predicted_binary_valence_score_knn = knn_clf_val_binary.predict(eeg_test)
cm = confusion_matrix(y_val_binary_test, y_predicted_binary_valence_score_knn)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Binary VALENCE using KNN: ")
print(accuracy_score(y_val_binary_test, y_predicted_binary_valence_score_knn)*100)

[[1961    1]
 [   1 1844]]
Accuracy score FOR SUBJECT 1 of Binary VALENCE using KNN: 
99.94746519569215


In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_val_binary_test, y_predicted_binary_valence_score_knn, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      1962
     class 1       1.00      1.00      1.00      1845

    accuracy                           1.00      3807
   macro avg       1.00      1.00      1.00      3807
weighted avg       1.00      1.00      1.00      3807



11.2.1 KNN Binary [0,1] **AROUSAL**


In [ ]:
eeg_train, eeg_test, y_ars_binary_train, y_ars_binary_test = train_test_split(datas_one, y_ars_binary, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVC Classifier on BINARY AROUSAL for ONE Subject
knn_clf_ars_binary = KNeighborsClassifier(n_neighbors=1)
knn_clf_ars_binary.fit(eeg_train, y_ars_binary_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
y_predicted_binary_arousal_score_knn = knn_clf_ars_binary.predict(eeg_test)
cm = confusion_matrix(y_ars_binary_test, y_predicted_binary_arousal_score_knn)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Binary AROUSAL using KNN: ")
print(accuracy_score(y_ars_binary_test, y_predicted_binary_arousal_score_knn)*100)

[[1490    2]
 [   0 2315]]
Accuracy score FOR SUBJECT 1 of Binary AROUSAL using KNN: 
99.94746519569215


In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(y_ars_binary_test, y_predicted_binary_arousal_score_knn, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      1492
     class 1       1.00      1.00      1.00      2315

    accuracy                           1.00      3807
   macro avg       1.00      1.00      1.00      3807
weighted avg       1.00      1.00      1.00      3807



11.2.2 KNN Tertiary [0,1,2]

11.2.2.1 KNN Tertiary [0,1,2] **VALENCE**

In [ ]:
eeg_train, eeg_test, y_val_tertiary_train, y_val_tertiary_test = train_test_split(datas_one, y_val_ter, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train KNN Classifier on TERTIARY VALENCE for ONE Subject
knn_clf_val_tertiary = KNeighborsClassifier(n_neighbors=1)
knn_clf_val_tertiary.fit(eeg_train, y_val_tertiary_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
y_predicted_tertiary_valence_score_knn = knn_clf_val_tertiary.predict(eeg_test)
cm = confusion_matrix(y_val_tertiary_test, y_predicted_tertiary_valence_score_knn)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Tertiary Valence using KNN: ")
print(accuracy_score(y_val_tertiary_test, y_predicted_tertiary_valence_score_knn)*100)

[[1549    0    1]
 [   0  491    0]
 [   1    0 1765]]
Accuracy score FOR SUBJECT 1 of Tertiary Valence using KNN: 
99.94746519569215


In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_val_tertiary_test, y_predicted_tertiary_valence_score_knn, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      1550
     class 1       1.00      1.00      1.00       491
     class 2       1.00      1.00      1.00      1766

    accuracy                           1.00      3807
   macro avg       1.00      1.00      1.00      3807
weighted avg       1.00      1.00      1.00      3807



11.2.2.1 KNN Tertiary [0,1,2] **AROUSAL**

In [ ]:
eeg_train, eeg_test, y_ars_tertiary_train, y_ars_tertiary_test = train_test_split(datas_one, y_ars_ter, test_size = 0.2, random_state = 42)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train KNN Classifier on TERTIARY AROUSAL for ONE Subject
knn_clf_ars_tertiary = KNeighborsClassifier(n_neighbors=1)
knn_clf_ars_tertiary.fit(eeg_train, y_ars_tertiary_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [ ]:
y_predicted_tertiary_arousal_score_knn = knn_clf_ars_tertiary.predict(eeg_test)
cm = confusion_matrix(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_knn)
print(cm)
print("Accuracy score FOR SUBJECT 1 of Tertiary AROUSAL using KNN: ")
print(accuracy_score(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_knn)*100)

[[1092    0    1]
 [   0  489    1]
 [   0    0 2224]]
Accuracy score FOR SUBJECT 1 of Tertiary AROUSAL using KNN: 
99.94746519569215


In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_ars_tertiary_test, y_predicted_tertiary_arousal_score_knn, target_names=target_names))

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      1093
     class 1       1.00      1.00      1.00       490
     class 2       1.00      1.00      1.00      2224

    accuracy                           1.00      3807
   macro avg       1.00      1.00      1.00      3807
weighted avg       1.00      1.00      1.00      3807



12. REGRESSION PROBLEMS


*   12.1 SVR (Support Vector Regression)
*   12.2 RANDOM FOREST REGRESSION 
*   12.3 AdaBoost



12.1 SVR (Support Vector Regression)

12.1.1 SVR **VALENCE** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_val_ratings_train, y_val_ratings_test = train_test_split(datas_one, y_val_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
y_val_ratings_train = np.ravel(y_val_ratings_train)
#y_ars_train = np.ravel(y_ars_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
y_val_ratings_test = np.ravel(y_val_ratings_test)
#y_ars_test = np.ravel(y_ars_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)


In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVR Classifier on VALENCE for ONE Subject
svr_clf_val = svm.SVR()
svr_clf_val.fit(eeg_train, y_val_ratings_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#Predict VALENCE with the SVR classifier
y_predicted_valence_score_svr = svr_clf_val.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR SUBJECT 1 FOR VALENCE SCORE WITH SVR-----------")
print("Explained variance score: ", explained_variance_score(y_val_ratings_test, y_predicted_valence_score_svr))
print("Mean absolute error: ", mean_absolute_error(y_val_ratings_test, y_predicted_valence_score_svr))
print("Mean squared error: ", mean_squared_error(y_val_ratings_test, y_predicted_valence_score_svr))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_val_ratings_test, y_predicted_valence_score_svr))
print("R2 score: ", r2_score(y_val_ratings_test, y_predicted_valence_score_svr))

----------METRICS FOR SUBJECT 1 FOR VALENCE SCORE WITH SVR-----------
Explained variance score:  0.7844663624286095
Mean absolute error:  0.826714415613305
Mean squared error:  1.3595882128188699
Mean squared logarithmic error:  0.04985762432789213
R2 score:  0.7842846731051814


12.1.2 SVR **AROUSAL** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_ars_ratings_train, y_ars_ratings_test = train_test_split(datas_one, y_ars_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
#y_val_ratings_train = np.ravel(y_val_ratings_train)
y_ars_ratings_train = np.ravel(y_ars_ratings_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
#y_val_ratings_test = np.ravel(y_val_ratings_test)
y_ars_ratings_test = np.ravel(y_ars_ratings_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
#Train SVR Classifier on VALENCE for ONE Subject
svr_clf_ars = svm.SVR()
svr_clf_ars.fit(eeg_train, y_ars_ratings_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
#Predict AROUSAL with the SVR classifier
y_predicted_arousal_score_svr = svr_clf_ars.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR SUBJECT 1FOR AROUSAL SCORE WITH SVR-----------")
print("Explained variance score: ", explained_variance_score(y_ars_ratings_test, y_predicted_arousal_score_svr))
print("Mean absolute error: ", mean_absolute_error(y_ars_ratings_test, y_predicted_arousal_score_svr))
print("Mean squared error: ", mean_squared_error(y_ars_ratings_test, y_predicted_arousal_score_svr))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_ars_ratings_test, y_predicted_arousal_score_svr))
print("R2 score: ", r2_score(y_ars_ratings_test, y_predicted_arousal_score_svr))

----------METRICS FOR SUBJECT 1FOR AROUSAL SCORE WITH SVR-----------
Explained variance score:  0.8126418111902998
Mean absolute error:  0.6322885575800817
Mean squared error:  0.8841825253278518
Mean squared logarithmic error:  0.03178824220346806
R2 score:  0.8120868692094585


12.2 Random Forest Regression

12.2.1 Random Forest **VALENCE** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_val_ratings_train, y_val_ratings_test = train_test_split(datas_one, y_val_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
y_val_ratings_train = np.ravel(y_val_ratings_train)
#y_ars_train = np.ravel(y_ars_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
y_val_ratings_test = np.ravel(y_val_ratings_test)
#y_ars_test = np.ravel(y_ars_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
Val_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Val_R.fit(eeg_train, y_val_ratings_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=512, n_jobs=6, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
#Predict VALENCE with the RANDOM FOREST classifier
y_predicted_valence_score_rf = Val_R.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR SUBJECT 1 FOR VALENCE SCORE WITH RANDOM FOREST-----------")
print("Explained variance score: ", explained_variance_score(y_val_ratings_test, y_predicted_valence_score_rf))
print("Mean absolute error: ", mean_absolute_error(y_val_ratings_test, y_predicted_valence_score_rf))
print("Mean squared error: ", mean_squared_error(y_val_ratings_test, y_predicted_valence_score_rf))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_ars_ratings_test, y_predicted_valence_score_rf))
print("R2 score: ", r2_score(y_val_ratings_test, y_predicted_valence_score_rf))

----------METRICS FOR SUBJECT 1 FOR VALENCE SCORE WITH RANDOM FOREST-----------
Explained variance score:  0.8005633968072984
Mean absolute error:  0.8931341268961721
Mean squared error:  1.2570053375695975
Mean squared logarithmic error:  0.2078235854902752
R2 score:  0.8005607030527544


12.2.1 Random Forest **AROUSAL** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_ars_ratings_train, y_ars_ratings_test = train_test_split(datas_one, y_ars_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
#y_val_ratings_train = np.ravel(y_val_ratings_train)
y_ars_ratings_train = np.ravel(y_ars_ratings_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
#y_val_ratings_test = np.ravel(y_val_ratings_test)
y_ars_ratings_test = np.ravel(y_ars_ratings_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
Ars_R = RandomForestRegressor(n_estimators=512, n_jobs=6)
Ars_R.fit(eeg_train, y_ars_ratings_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=512, n_jobs=6, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
#Predict AROUSAL with the RANDOM FOREST classifier
y_predicted_arousal_score_rf = Ars_R.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR AROUSAL SCORE WITH RANDOM FOREST-----------")
print("Explained variance score: ", explained_variance_score(y_ars_ratings_test, y_predicted_arousal_score_rf))
print("Mean absolute error: ", mean_absolute_error(y_ars_ratings_test, y_predicted_arousal_score_rf))
print("Mean squared error: ", mean_squared_error(y_ars_ratings_test, y_predicted_arousal_score_rf))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_ars_ratings_test, y_predicted_arousal_score_rf))
print("R2 score: ", r2_score(y_ars_ratings_test, y_predicted_arousal_score_rf))

----------METRICS FOR AROUSAL SCORE WITH RANDOM FOREST-----------
Explained variance score:  0.8103141125304344
Mean absolute error:  0.7004649843216428
Mean squared error:  0.892583758113848
Mean squared logarithmic error:  0.031166730822049314
R2 score:  0.8103013759316635


12.3 AdaBoost

12.3.1 AdaBoost **VALENCE** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_val_ratings_train, y_val_ratings_test = train_test_split(datas_one, y_val_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
y_val_ratings_train = np.ravel(y_val_ratings_train)
#y_ars_train = np.ravel(y_ars_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
y_val_ratings_test = np.ravel(y_val_ratings_test)
#y_ars_test = np.ravel(y_ars_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
AdaBoost_Val = AdaBoostRegressor(n_estimators=5000, learning_rate=0.01)
AdaBoost_Val.fit(eeg_train, y_val_ratings_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=5000, random_state=None)

In [ ]:
#Predict VALENCE with the RANDOM FOREST classifier
y_predicted_valence_score_adaboost = AdaBoost_Val.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR VALENCE SCORE WITH ADABOOST-----------")
print("Explained variance score: ", explained_variance_score(y_val_ratings_test, y_predicted_valence_score_adaboost))
print("Mean absolute error: ", mean_absolute_error(y_val_ratings_test, y_predicted_valence_score_adaboost))
print("Mean squared error: ", mean_squared_error(y_val_ratings_test, y_predicted_valence_score_adaboost))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_ars_ratings_test, y_predicted_valence_score_adaboost))
print("R2 score: ", r2_score(y_val_ratings_test, y_predicted_valence_score_adaboost))

----------METRICS FOR VALENCE SCORE WITH ADABOOST-----------
Explained variance score:  0.19700221783630822
Mean absolute error:  2.092800053762886
Mean squared error:  5.064815183882615
Mean squared logarithmic error:  0.16419657729206608
R2 score:  0.19640501973178381


12.3.2 AdaBoost **AROUSAL** Ratings [1-9]

In [ ]:
eeg_train, eeg_test, y_ars_ratings_train, y_ars_ratings_test = train_test_split(datas_one, y_ars_one, test_size = 0.2, random_state = 42)

In [ ]:
#Let's normalize the training and testing eeg and labels
eeg_train = normalize(eeg_train)
#y_val_ratings_train = np.ravel(y_val_ratings_train)
y_ars_ratings_train = np.ravel(y_ars_ratings_train)
#y_lik_train = np.ravel(y_lik_train)
#y_dom_train = np.ravel(y_dom_train)
#######
eeg_test = normalize(eeg_test)
#y_val_ratings_test = np.ravel(y_val_ratings_test)
y_ars_ratings_test = np.ravel(y_ars_ratings_test)
#y_lik_test = np.ravel(y_lik_test)
#y_dom_test = np.ravel(y_dom_test)

In [ ]:
#Feature Scaling
sc = StandardScaler()
eeg_train = sc.fit_transform(eeg_train)
eeg_test = sc.transform(eeg_test)

In [ ]:
AdaBoost_Ars = AdaBoostRegressor(n_estimators=5000, learning_rate=0.01)
AdaBoost_Ars.fit(eeg_train, y_ars_ratings_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=5000, random_state=None)

In [ ]:
#Predict AROUSAL with the RANDOM FOREST classifier
y_predicted_arousal_score_adaboost = AdaBoost_Ars.predict(eeg_test) #M is the X_test, L is the Y_test
#print(predicted_valence_score_svr)
#Compute Metrics of the SVR classifier for VALENCE
#testing (eeg_test, y_val_ratings_test, svr_clf_val)
print("----------METRICS FOR AROUSAL SCORE WITH ADABOOST-----------")
print("Explained variance score: ", explained_variance_score(y_ars_ratings_test, y_predicted_arousal_score_adaboost))
print("Mean absolute error: ", mean_absolute_error(y_ars_ratings_test, y_predicted_arousal_score_adaboost))
print("Mean squared error: ", mean_squared_error(y_ars_ratings_test, y_predicted_arousal_score_adaboost))
print("Mean squared logarithmic error: ", mean_squared_log_error(y_ars_ratings_test, y_predicted_arousal_score_adaboost))
print("R2 score: ", r2_score(y_ars_ratings_test, y_predicted_arousal_score_adaboost))

----------METRICS FOR AROUSAL SCORE WITH ADABOOST-----------
Explained variance score:  0.21229272938134625
Mean absolute error:  1.7310914707070477
Mean squared error:  3.713717006529609
Mean squared logarithmic error:  0.1139899169307968
R2 score:  0.21073288650633093
